In [8]:
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd

from transformers import BertTokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
trn_df = pd.read_csv('../inputs/origin/train.csv')
trn_df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [14]:
invalid_textIDs = []

for i, row in tqdm(list(trn_df[trn_df.text.notnull()].iterrows())):
        text_output = tokenizer.encode_plus(
            text=row['text'],
            text_pair=None,
            add_special_tokens=True,
            max_length=250,
            pad_to_max_length=True,
            return_tensor='pt',
            return_token_type_ids=False,
            return_attention_mask=True,
        )
        selected_text_output = tokenizer.encode_plus(
            text=row['selected_text'],
            text_pair=None,
            add_special_tokens=False,
            max_length=250,
            pad_to_max_length=False,
            return_tensor='pt',
            return_token_type_ids=False,
            return_attention_mask=False,
        )

        # allign labels for segmentation
        input_ids = text_output['input_ids']
        sel_input_ids = selected_text_output['input_ids']
        matched_cnt = len([i for i in input_ids[:len(sel_input_ids)]
                           if i in sel_input_ids])
        best_matched_cnt = len([i for i in input_ids[:len(sel_input_ids)]
                                if i in sel_input_ids])
        best_matched_i = 0
        # for i in range(0, len(input_ids)):
        for i in range(0, len(input_ids) - len(sel_input_ids)):
            head_input_id_i = input_ids[i]
            tail_input_id_i = input_ids[i + len(sel_input_ids)]
            if head_input_id_i in sel_input_ids:
                matched_cnt -= 1
            if tail_input_id_i in sel_input_ids:
                matched_cnt += 1
            if matched_cnt < 0:
                raise Exception('invalid logic')

            if best_matched_cnt < matched_cnt:
                best_matched_cnt = matched_cnt
                best_matched_i = i + 1   # $BH4$$$?;~$NOC$J$N$G(B
            if best_matched_cnt == len(sel_input_ids):
                break
            # if input_id_i == sel_input_ids[0]:
            # if input_id_i in sel_input_ids:
            #     temp_matched_len = 0
            #     for t_1, t_2 in zip(
            #             input_ids[i:i + len(sel_input_ids)], sel_input_ids):
            #         if t_1 == t_2:
            #             temp_matched_len += 1
            #     if best_matched_len < temp_matched_len:
            #         best_matched_len = temp_matched_len
            #         best_matched_i = i
            #     if best_matched_len == len(sel_input_ids):
            #         break
        if best_matched_cnt == 0:    
            invalid_textIDs.append(row['textID'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
invalid_textIDs

['af3fed7fc3',
 'a54d3c2825',
 'ee9df322d1',
 '31fa81e0ae',
 'eadba04b8a',
 '795de6c2f5',
 'c4ffb519a5',
 '95593ecedb',
 'ee5eb5337b',
 'd5b33ff5f4',
 '792063a20e',
 'cecd8f1042',
 '43e6d9aeaa',
 '9df7f02404',
 '780c673bca',
 'fbaace1664',
 '43887b9d65',
 '78a714e8bb',
 'a4ba901f22',
 '12f21c8f19',
 '3e775363a1',
 '1e0fefceaf',
 'f6b45a61d5',
 'a65d96c8c9',
 'f2eb0b5e90',
 '96ff964db0',
 'ce0025d376',
 '1950e8ff99',
 'e5d31d8d38',
 'df398a774e',
 'd2a221697e',
 '633e9e07ff',
 'f9766da79f',
 'd0ecfc8c28',
 '8b4a1efb7e',
 'c573f1463f',
 'c155efab1b',
 '75bf000837',
 '23b8ac0826',
 '00f02e92d6',
 '4b3fd4ec22',
 'f1dadac45b',
 '466f707c18',
 '142f9472fc',
 'd635eff196',
 '0e0ea93f6e',
 'debbd12acd',
 'cd5989172a',
 'b91963e312',
 '57a498a841',
 'db01ffbc9f',
 '99f9b82224',
 '09d0f8f088',
 '8a297e9460',
 '2e3afc2394',
 '61306b5f1e',
 '367029df78',
 '24d37f9ba7',
 '7b11a02202',
 'a11ddb9e58',
 '30a1e8c2b4',
 'e149ebd3a1']

In [16]:
invalid_labeled_trn_df = trn_df.query(f'textID in {invalid_textIDs}')
invalid_labeled_trn_df

,textID,text,selected_text,sentiment
18,af3fed7fc3,is back home now gonna miss every one,onna,negative
309,a54d3c2825,"I know It was worth a shot, though!",as wort,positive
492,ee9df322d1,"Sorry, we`ll try to keep it down.",Sorr,negative
809,31fa81e0ae,"Yes, you should go see Star Trek! It`s sooo...",ch fu,positive
1376,eadba04b8a,that would mean me babe! but **** it my name...,IN COO,positive
...,...,...,...,...
26830,24d37f9ba7,my bracelet broke today too.,elet br,negative
26976,7b11a02202,We were all set to get a room last Saturday...,or missin,negative
27121,a11ddb9e58,The geographY was an exam today!But turned out...,ry nervou,negative
27209,30a1e8c2b4,Guys ( ) I know! My ability to read time te...,es faile,negative


In [24]:
annot_dicts = []

for i, row in invalid_labeled_trn_df.iterrows():
    print(f' ===== {row["textID"]} : {row["sentiment"]} ===== ')
    print(f'text: {row["text"]}')
    print(f'text: {row["selected_text"]}')
    print()
    annot_dicts.append({'textID': row['textID'], 'guchio_selected_text': None})

 ===== af3fed7fc3 : negative ===== 
text: is back home now      gonna miss every one
text: onna

 ===== a54d3c2825 : positive ===== 
text:  I know   It was worth a shot, though!
text: as wort

 ===== ee9df322d1 : negative ===== 
text:    Sorry, we`ll try to keep it down.
text: Sorr

 ===== 31fa81e0ae : positive ===== 
text:   Yes, you should go see Star Trek!  It`s sooooo much fun!
text: ch fu

 ===== eadba04b8a : positive ===== 
text:   that would mean me babe! but **** it my name  is SUPER FREAKIN COOL! I give myself a pass! HA!
text: IN COO

 ===== 795de6c2f5 : positive ===== 
text:  ooh ouch!   lovely weather but ****! cover up sugar! x
text: lovel

 ===== c4ffb519a5 : positive ===== 
text: I don`t feel any pressure right now...    Happy Mother`s Day people...
text: Happ

 ===== 95593ecedb : positive ===== 
text:  Cool Sound!! Luv 'Drama', 'Love Game' doesn`t work on ur myspace...    Wish Good Luck!! XoXo from Spain
text: ish Go

 ===== ee5eb5337b : negative ===== 
text: Heading to

In [40]:
invalid_labels_df = pd.DataFrame([{'guchio_selected_text': 'miss', 'textID': 'af3fed7fc3'},
 {'guchio_selected_text': 'worth', 'textID': 'a54d3c2825'},
 {'guchio_selected_text': 'Sorry', 'textID': 'ee9df322d1'},
 {'guchio_selected_text': 'fun', 'textID': '31fa81e0ae'},
 {'guchio_selected_text': 'COOL', 'textID': 'eadba04b8a'},
 {'guchio_selected_text': 'lovely', 'textID': '795de6c2f5'},
 {'guchio_selected_text': 'Happy', 'textID': 'c4ffb519a5'},
 {'guchio_selected_text': 'Good Luck', 'textID': '95593ecedb'},
 {'guchio_selected_text': 'terribly sad', 'textID': 'ee5eb5337b'},
 {'guchio_selected_text': 'dirty', 'textID': 'd5b33ff5f4'},
 {'guchio_selected_text': 'Thanks', 'textID': '792063a20e'},
 {'guchio_selected_text': 'good', 'textID': 'cecd8f1042'},
 {'guchio_selected_text': 'Thanks', 'textID': '43e6d9aeaa'},
 {'guchio_selected_text': 'PhootoBoothingisFunForBunny', 'textID': '9df7f02404'},
 {'guchio_selected_text': 'good', 'textID': '780c673bca'},
 {'guchio_selected_text': 'sounds amazing', 'textID': 'fbaace1664'},
 {'guchio_selected_text': 'Love', 'textID': '43887b9d65'},
 {'guchio_selected_text': 'Bored', 'textID': '78a714e8bb'},
 {'guchio_selected_text': 'really good', 'textID': 'a4ba901f22'},
 {'guchio_selected_text': 'LOL', 'textID': '12f21c8f19'},
 {'guchio_selected_text': 'sucks', 'textID': '3e775363a1'},
 {'guchio_selected_text': 'thanks', 'textID': '1e0fefceaf'},
 {'guchio_selected_text': 'probably better', 'textID': 'f6b45a61d5'},
 {'guchio_selected_text': 'love and kindness', 'textID': 'a65d96c8c9'},
 {'guchio_selected_text': 'best', 'textID': 'f2eb0b5e90'},
 {'guchio_selected_text': 'hate', 'textID': '96ff964db0'},
 {'guchio_selected_text': 'weird', 'textID': 'ce0025d376'},
 {'guchio_selected_text': 'good', 'textID': '1950e8ff99'},
 {'guchio_selected_text': 'great', 'textID': 'e5d31d8d38'},
 {'guchio_selected_text': 'have fun', 'textID': 'df398a774e'},
 {'guchio_selected_text': 'was clean', 'textID': 'd2a221697e'},
 {'guchio_selected_text': 'miss', 'textID': '633e9e07ff'},
 {'guchio_selected_text': 'nice', 'textID': 'f9766da79f'},
 {'guchio_selected_text': 'absolutely amazing', 'textID': 'd0ecfc8c28'},
 {'guchio_selected_text': 'Happy', 'textID': '8b4a1efb7e'},
 {'guchio_selected_text': 'HAPPY', 'textID': 'c573f1463f'},
 {'guchio_selected_text': 'Bunny', 'textID': 'c155efab1b'},
 {'guchio_selected_text': 'Sorry', 'textID': '75bf000837'},
 {'guchio_selected_text': 'thanks', 'textID': '23b8ac0826'},
 {'guchio_selected_text': 'great', 'textID': '00f02e92d6'},
 {'guchio_selected_text': 'sad', 'textID': '4b3fd4ec22'},
 {'guchio_selected_text': 'really cool', 'textID': 'f1dadac45b'},
 {'guchio_selected_text': 'exciting', 'textID': '466f707c18'},
 {'guchio_selected_text': '****', 'textID': '142f9472fc'},
 {'guchio_selected_text': 'AMAZING', 'textID': 'd635eff196'},
 {'guchio_selected_text': 'the best', 'textID': '0e0ea93f6e'},
 {'guchio_selected_text': 'just mean', 'textID': 'debbd12acd'},
 {'guchio_selected_text': 'Sorry', 'textID': 'cd5989172a'},
 {'guchio_selected_text': 'harder', 'textID': 'b91963e312'},
 {'guchio_selected_text': 'just joking', 'textID': '57a498a841'},
 {'guchio_selected_text': 'thanks', 'textID': 'db01ffbc9f'},
 {'guchio_selected_text': 'Happy', 'textID': '99f9b82224'},
 {'guchio_selected_text': 'wow', 'textID': '09d0f8f088'},
 {'guchio_selected_text': 'just ignore', 'textID': '8a297e9460'},
 {'guchio_selected_text': 'tired', 'textID': '2e3afc2394'},
 {'guchio_selected_text': 'love', 'textID': '61306b5f1e'},
 {'guchio_selected_text': 'glad', 'textID': '367029df78'},
 {'guchio_selected_text': 'broke', 'textID': '24d37f9ba7'},
 {'guchio_selected_text': 'missing', 'textID': '7b11a02202'},
 {'guchio_selected_text': 'nervous', 'textID': 'a11ddb9e58'},
 {'guchio_selected_text': 'failed', 'textID': '30a1e8c2b4'},
 {'guchio_selected_text': 'appreciate', 'textID': 'e149ebd3a1'}])

invalid_labels_df.head()

,guchio_selected_text,textID
0,miss,af3fed7fc3
1,worth,a54d3c2825
2,Sorry,ee9df322d1
3,fun,31fa81e0ae
4,COOL,eadba04b8a


In [41]:
!mkdir ../inputs/nes_info

In [42]:
invalid_labels_df.to_csv('../inputs/nes_info/invalid_labels.csv', index=False)

In [31]:
tokenizer.decode(tokenizer.encode_plus('http://twitpic.com/4wc1y - Cracking myself more & more up..  PhootoBoothingisFunForBunny Volumen Eins!')['input_ids'])

'[CLS] http : / / twitpic. com / 4wc1y - cracking myself more & more up.. phootoboothingisfunforbunny volumen eins! [SEP]'

In [35]:
tokenizer.convert_ids_to_tokens(tokenizer.encode_plus('star wars ............ is **** BOO??? i wanna do your job HAND IT OVER  u can act as me at my high school   LOL')['input_ids'])

['[CLS]', 'star', 'wars', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', 'is', '*', '*', '*', '*', 'boo', '?', '?', '?', 'i', 'wanna', 'do', 'your', 'job', 'hand', 'it', 'over', 'u', 'can', 'act', 'as', 'me', 'at', 'my', 'high', 'school', 'lo', '##l', '[SEP]']

In [29]:
dir(tokenizer)

['NO_PAD_TOKEN_FOR_BATCH_MSG', 'SPECIAL_TOKENS_ATTRIBUTES', 'UNEVEN_SEQUENCES_FOR_BATCH_MSG', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_additional_special_tokens', '_bos_token', '_cls_token', '_convert_id_to_token', '_convert_token_to_id', '_convert_token_to_id_with_added_voc', '_eos_token', '_from_pretrained', '_mask_token', '_pad_token', '_pad_token_type_id', '_sep_token', '_tokenize', '_unk_token', 'add_special_tokens', 'add_tokens', 'added_tokens_decoder', 'added_tokens_encoder', 'additional_special_tokens', 'additional_special_tokens_ids', 'all_special_ids', 'all_special_tokens', 'basic_tokenizer', 'batch_encode_plus', 'bos_token', 'bos_token_id', 'build_inputs_with_speci

In [23]:
from pprint import pprint
pprint(annot_dicts)

[{'guchio_selected_text': None, 'textID': 'af3fed7fc3'},
 {'guchio_selected_text': None, 'textID': 'a54d3c2825'},
 {'guchio_selected_text': None, 'textID': 'ee9df322d1'},
 {'guchio_selected_text': None, 'textID': '31fa81e0ae'},
 {'guchio_selected_text': None, 'textID': 'eadba04b8a'},
 {'guchio_selected_text': None, 'textID': '795de6c2f5'},
 {'guchio_selected_text': None, 'textID': 'c4ffb519a5'},
 {'guchio_selected_text': None, 'textID': '95593ecedb'},
 {'guchio_selected_text': None, 'textID': 'ee5eb5337b'},
 {'guchio_selected_text': None, 'textID': 'd5b33ff5f4'},
 {'guchio_selected_text': None, 'textID': '792063a20e'},
 {'guchio_selected_text': None, 'textID': 'cecd8f1042'},
 {'guchio_selected_text': None, 'textID': '43e6d9aeaa'},
 {'guchio_selected_text': None, 'textID': '9df7f02404'},
 {'guchio_selected_text': None, 'textID': '780c673bca'},
 {'guchio_selected_text': None, 'textID': 'fbaace1664'},
 {'guchio_selected_text': None, 'textID': '43887b9d65'},
 {'guchio_selected_text': None,